<a href="https://colab.research.google.com/github/tomneutens/notebooks/blob/main/Kopie_van_LLAMA_2_Fine_Tuning_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

In [4]:
import json
import itertools
import numpy as np
from datasets import Dataset
training_data = []
with open("sample_data/pc_dataset.json", 'r', encoding="utf-8") as file:
  training_data = json.load(file)
training_data = {"text": list(itertools.chain.from_iterable([elem["conversations"] for elem in training_data]))}
training_data = Dataset.from_dict(training_data, split="train")
print(training_data)


Dataset({
    features: ['text'],
    num_rows: 305
})


In [5]:
# Dataset
# data_name = "mlabonne/guanaco-llama2-1k"
# training_data = load_dataset(data_name, split="train")

# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-dwengo-q-a" #You can give it your own name

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
training_data.shape
print(training_data[10])

{'text': '<s>[INST]  Hoe kunnen we het energieverbruik van deze digitale applicaties in perspectief plaatsen voor de leerlingen, bijvoorbeeld door te vergelijken met dagelijkse activiteiten of huishoudelijke apparaten?\n\n [/INST]  Om het energieverbruik van digitale applicaties in perspectief te plaatsen, kunnen we vergelijkingen maken met apparaten die leerlingen kennen uit hun dagelijks leven. Bijvoorbeeld, de Apple M3 chip gebruikt 30W per uur, wat overeenkomt met een kleine LED-lamp. We kunnen het jaarlijks verbruik van Google datacenters vergelijken met het aantal gemiddeld verbruik voor een Vlaams gezin, om te tonen hoeveel gezinnen daarmee van energie voorzien zouden kunnen worden.\n\n \n\n Meer informatie kan je vinden in het leerpad <lo_info>leerlijn_intro_klasgesprek;nl;1</lo_info> <lp_info><title>Leerlijn microcontrollers 3. Introductie</title><hruid>pc_leerlijn_introductie</hruid><language>nl</language></lp_info> </s>'}


In [7]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/305 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.799400
50,1.043300
75,0.895300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [8]:
print(fine_tuning.state.log_history)

[{'loss': 1.7994, 'learning_rate': 0.0002, 'epoch': 0.32, 'step': 25}, {'loss': 1.0433, 'learning_rate': 0.0002, 'epoch': 0.65, 'step': 50}, {'loss': 0.8953, 'learning_rate': 0.0002, 'epoch': 0.97, 'step': 75}, {'train_runtime': 279.6862, 'train_samples_per_second': 1.091, 'train_steps_per_second': 0.275, 'total_flos': 1750480202391552.0, 'train_loss': 1.2340371330063065, 'epoch': 1.0, 'step': 77}]


In [16]:
# Generate Text
query = "Wat zijn digitale en analoge pinnen?"
text_gen = pipeline(task="text-generation", model=fine_tuning.model, tokenizer=llama_tokenizer, max_length=1024)
output = text_gen(f"<s>[INST] {query} [/INST]", )
print(output[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

<s>[INST] Wat zijn digitale en analoge pinnen? [/INST] 

Digitale pinnen zijn pinnen die een digitale signalen kunnen verwerken, wat betekent dat ze een eenvoudige 0 of 1-waarde kunnen versturen. Analoge pinnen zijn pinnen die een analoge signalen kunnen verwerken, wat betekent dat ze een continu signalen kunnen versturen.

 

 Meer informatie kan je vinden in het leerpad <lo_info>leerlijn_invoer_verwerking_uitvoer_analoge_invoer;nl;1</lo_info> <lp_info><title>Leerlijn microcontrollers 2. invoer en uitvoer</title><hruid>pc_leerlijn_invoer_verwerking_uitvoer</hruid><language>nl</language></lp_info>

 

 Gebaseerd op het leerpad <lp_info><title>Leerlijn microcontrollers 2. invoer en uitvoer</title><hruid>pc_leerlijn_invoer_verwerking_uitvoer</hruid><language>nl</language></lp_info>

 

 Voor meer informatie over digitale en analoge pinnen, kunt u het leerpad <lp_info><title>Leerlijn microcontrollers 1. basisprincipes</title><hruid>pc_leerlijn_basisprincipes_digitale_elektronica;nl;1</hru

In [15]:
query = "Heeft dwengo lesmateriaal over de wet van ohm?"
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

<s>[INST] Heeft dwengo lesmateriaal over de wet van ohm? [/INST]  Ja, Dwengo biedt lesmateriaal over de wet van ohm. everybody.dwengo.com/lesmateriaal/wet_van_ohm_lesmateriaal_oplossing_basis_elektronica_programmeren_met_python_3_1.html

 Meer informatie kan je vinden in het leerpad <lo_info>leerlijn_invoer_verwerking_uitvoer_analoge_invoer_oplossing;nl;1</lo_info> <lp_info><title>Leerlijn microcontrollers 2. invoer en uitvoer</title><hruid>pc_leerlijn_invoer_verwerking_uitvoer</hruid><language>nl</language></lp_info> <lp_info_language><title>Leerlijn microcontrollers 2. invoer en uitvoer</title><hruid>pc_leerlijn_invoer_verwerking_uitvoer</hruid><language>nl</language></lp_info_language>

 # leerlijn # microcontrollers # invoer en


In [ ]:
print(output)